In [1]:
import pandas as pd
# from IPython.display import display
pd.set_option("display.max_colwidth", None)

## Helper Functions

In [2]:
def save_as_csv(dataframe, file_name):
    import datetime
    date_info = datetime.datetime.now().strftime("%d-%m-%y")
    dataframe.to_csv(f"scraping_analysis/{date_info}_{file_name}.csv", index=False)

## Code

In [3]:
df = pd.read_json("scrape_archive/full_scrape_2.json")

In [4]:
# wie viele in jeder spalte ausser "url", null oder "" enthalten, und im string der spalte "url" das wort "kunst" vorkommt
df.loc[
    (
        (df.loc[:, ~df.columns.isin(["url"])] == "") |
        (df.loc[:, ~df.columns.isin(["url"])].isnull())
    ).all(axis=1)
    & df["url"].str.contains("https")
].shape[0]

66

In [9]:
def count_string_occurences_in_url(pandas_df):
    things_to_check = [
        "https",
        "kunst",
        "generalagenturen",
        "jobs",
        "report",
        "vcard",
        "mobiliare",
        "mobiliere",
        "hub",
        "medienmitteilungen",
        "gallerySlide",
    ]  # https to get 100%
    results = []
    for item in things_to_check:
        item_count = pandas_df[pandas_df["url"].str.contains(item)].shape[0]
        pct = round(item_count / pandas_df.shape[0] * 100, 2)
        count_empty = pandas_df.loc[
            ((pandas_df.loc[:, ~pandas_df.columns.isin(["url"])] == "")
                | (pandas_df.loc[:, ~pandas_df.columns.isin(["url"])].isnull())
            ).all(axis=1)
            & pandas_df["url"].str.contains(item)
        ].shape[0]
        count_empty_except_title = pandas_df.loc[
            ((pandas_df.loc[:, ~pandas_df.columns.isin(["url", "page_title"])] == "")
                | (pandas_df.loc[:, ~pandas_df.columns.isin(["url", "page_title"])].isnull())
            ).all(axis=1)
            & pandas_df["url"].str.contains(item)
        ].shape[0]
        results.append(
            {
                "item_name": item,
                "item_count": item_count,
                "%": pct,
                "empty_except_url": count_empty,
                # "all_empty_%": round(count_empty / item_count * 100, 2),
                "empty_cols_except_title_&_url": count_empty_except_title,
                # "title_&_url_empty_%": round(count_empty_except_title / item_count * 100, 2),

            }
        )
    df = pd.DataFrame(results).sort_values('item_count', ascending=[False])
    return df

In [10]:
count_string_occurences_in_url(df)
# save_as_csv(count_string_occurences_in_url(df), "evaluation_occurrences_of_words_in_url_after_first_scrape")

,item_name,item_count,%,empty_except_url,empty_cols_except_title_&_url
0,https,1741,100.00,66,280
9,medienmitteilungen,524,30.10,0,50
1,kunst,68,3.91,0,7
7,mobiliere,33,1.90,0,1
8,hub,17,0.98,13,13
6,mobiliare,6,0.34,0,0
4,report,1,0.06,1,1
2,generalagenturen,0,0.00,0,0
3,jobs,0,0.00,0,0
5,vcard,0,0.00,0,0


In [11]:
not_empty = df.loc[
    (
        (df.loc[:, ~df.columns.isin(["url"])] != "") |
        (df.loc[:, ~df.columns.isin(["url"])].notnull())
    ).all(axis=1)
    & ~((df["url"].str.contains("kunst")) | (df["url"].str.contains("vcard"))| (df["url"].str.contains("jobs"))| (df["url"].str.contains("report"))) 
]
not_empty.shape[0]
# save_as_csv(not_empty, "not_empty")

1672

In [189]:
empty_except_page_title = df.loc[
    (
        (df.loc[:, ~df.columns.isin(["url", "page_title"])] == "")
        | (df.loc[:, ~df.columns.isin(["url", "page_title"])].isnull())
    ).all(axis=1)
    # & ~(
    #     (df["url"].str.contains("kunst"))
    #     | (df["url"].str.contains("vcard"))
    #     | (df["url"].str.contains("jobs"))
    #     | (df["url"].str.contains("report"))
    #     | (df["url"].str.contains("gallerySlide"))
    # )
]
empty_except_page_title.shape[0]
# save_as_csv(empty_except_page_title, "empty_except_page_title_with_gallerySlide")

3305

In [172]:
https_df_everything_empty = df.loc[
    (
        (df.loc[:, ~df.columns.isin(["url"])] == "") |
        (df.loc[:, ~df.columns.isin(["url"])].isnull())
    ).all(axis=1)
    & ~((df["url"].str.contains("kunst")) | (df["url"].str.contains("vcard"))) 
]
https_df_everything_empty.shape[0]
display(https_df_everything_empty)
# save_as_csv(https_df_everything_empty, "all_empty_not_containing_url_or_vcard")

,url,page_title,sub_title,introduction,summary_box,content,accordion
28,https://www.mobiliar.ch/versicherungen-und-vorsorge/generalagenturen,None,None,None,,,
32,https://www.mobiliar.ch/hub/wohnen,None,None,None,,,
43,https://jobs.mobiliar.ch/Jobs/All?DesignID=00&lang=ger,None,None,None,,,
57,https://report.mobiliar.ch/2022/de/,None,None,None,,,
80,https://my.mobiliar.ch/portal-rwc/?language=de,None,None,None,,,
...,...,...,...,...,...,...,...
4707,https://www.mobiliar.ch/hub/wohnen/inspiration,None,None,None,,,
4709,https://www.mobiliar.ch/hub/wohnen/ratgeberartikel,None,None,None,,,
4710,https://www.mobiliar.ch/hub/wohnen/videoartikel,None,None,None,,,
4732,https://www.mobiliar.ch/hub/wohnen/unser-haus/unser-haus-making-of,None,None,None,,,


In [34]:
def count_not_null_or_empty_of_X(pandas_df):
    col_list = pandas_df.columns.values.tolist()
    results = []
    for x in col_list:
        x_count = pandas_df[pandas_df[x].notna() & (pandas_df[x] != "")].shape[0]
        pct = round(x_count/pandas_df.shape[0]*100, 2)
        results.append({
            'column_name': x,
            'is_not_empty_count': x_count,
            'percentage': pct
        })
    return pd.DataFrame(results)

In [35]:
count_not_null_or_empty_of_X(df)
# save_as_csv(count_not_null_or_empty_of_X(df), "count_not_empty_by_column")


,column_name,is_not_empty_count,percentage
0,url,1741,100.00
1,page_title,1675,96.21
2,sub_title,414,23.78
3,introduction,1242,71.34
4,summary_box,194,11.14
5,content,723,41.53
6,accordion,161,9.25
